# Initialize docker for google colab

In [7]:
%%bash
sudo apt-get update
sudo apt-get install -y ca-certificates curl gnupg
install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
chmod a+r /etc/apt/keyrings/docker.gpg
echo \
  "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://download.docker.com/linux/ubuntu jammy InRelease [48.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Err:2 https://download.docker.com/linux/ubuntu jammy InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 7EA0A9C3F273FCD8
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...


W: GPG error: https://download.docker.com/linux/ubuntu jammy InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 7EA0A9C3F273FCD8
E: The repository 'https://download.docker.com/linux/ubuntu jammy InRelease' is not signed.


Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
curl is already the newest version (7.81.0-1ubuntu1.10).
The following additional packages will be installed:
  dirmngr gnupg-l10n gnupg-utils gpg gpg-agent gpg-wks-client gpg-wks-server
  gpgconf gpgsm libassuan0 libksba8 libnpth0 pinentry-curses
Suggested packages:
  dbus-user-session libpam-systemd pinentry-gnome3 tor parcimonie xloadimage
  scdaemon pinentry-doc
The following NEW packages will be installed:
  dirmngr gnupg gnupg-l10n gnupg-utils gpg gpg-agent gpg-wks-client
  gpg-wks-server gpgconf gpgsm libassuan0 libksba8 libnpth0 pinentry-curses
0 upgraded, 14 newly installed, 0 to remove and 0 not upgraded.
Need to get 2,309 kB of archives.
After this operation, 5,837 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libassuan0 amd64 2.5.5-1build1 [38.2 kB]
Get:2 http://archive.

debconf: delaying package configuration, since apt-utils is not installed


Fetched 2,309 kB in 25s (93.3 kB/s)
Selecting previously unselected package libassuan0:amd64.
(Reading database ... 43028 files and directories currently installed.)
Preparing to unpack .../00-libassuan0_2.5.5-1build1_amd64.deb ...
Unpacking libassuan0:amd64 (2.5.5-1build1) ...
Selecting previously unselected package gpgconf.
Preparing to unpack .../01-gpgconf_2.2.27-3ubuntu2.1_amd64.deb ...
Unpacking gpgconf (2.2.27-3ubuntu2.1) ...
Selecting previously unselected package libksba8:amd64.
Preparing to unpack .../02-libksba8_1.6.0-2ubuntu0.2_amd64.deb ...
Unpacking libksba8:amd64 (1.6.0-2ubuntu0.2) ...
Selecting previously unselected package libnpth0:amd64.
Preparing to unpack .../03-libnpth0_1.6-3build2_amd64.deb ...
Unpacking libnpth0:amd64 (1.6-3build2) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.2.27-3ubuntu2.1_amd64.deb ...
Unpacking dirmngr (2.2.27-3ubuntu2.1) ...
Selecting previously unselected package gnupg-l10n.
Preparing to unpack .

chmod: changing permissions of '/etc/apt/keyrings/docker.gpg': Operation not permitted


In [11]:
%%bash
sudo apt update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...
Building dependency tree...
Reading state information...
All packages are up to date.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  apparmor dbus dbus-user-session dmsetup docker-ce-rootless-extras
  gir1.2-glib-2.0 iptables libapparmor1 libargon2-1 libcryptsetup12
  libdbus-1-3 libdevmapper1.02.1 libgirepository-1.0-1 libip4tc2 libip6tc2
  libjson-c5 libkmod2 libltdl7 libmnl0 libnetfilter-conntrack3 libnfnetlink0
  libnftnl11 libnss-systemd libpam-systemd libslirp0 libxtables12 netbase
  networkd-dispatcher pigz python3-dbus python3-gi slirp4netns systemd
  systemd-

debconf: delaying package configuration, since apt-utils is not installed


Fetched 118 MB in 3min 58s (497 kB/s)
Selecting previously unselected package libapparmor1:amd64.
(Reading database ... 43239 files and directories currently installed.)
Preparing to unpack .../0-libapparmor1_3.0.4-2ubuntu2.2_amd64.deb ...
Unpacking libapparmor1:amd64 (3.0.4-2ubuntu2.2) ...
Selecting previously unselected package libargon2-1:amd64.
Preparing to unpack .../1-libargon2-1_0~20171227-0.3_amd64.deb ...
Unpacking libargon2-1:amd64 (0~20171227-0.3) ...
Selecting previously unselected package libdevmapper1.02.1:amd64.
Preparing to unpack .../2-libdevmapper1.02.1_2%3a1.02.175-2.1ubuntu4_amd64.deb ...
Unpacking libdevmapper1.02.1:amd64 (2:1.02.175-2.1ubuntu4) ...
Selecting previously unselected package libjson-c5:amd64.
Preparing to unpack .../3-libjson-c5_0.15-3~ubuntu1.22.04.1_amd64.deb ...
Unpacking libjson-c5:amd64 (0.15-3~ubuntu1.22.04.1) ...
Selecting previously unselected package libcryptsetup12:amd64.
Preparing to unpack .../4-libcryptsetup12_2%3a2.4.3-1ubuntu1.1_amd64.d

In [34]:
%%script bash --bg --out docker_out --err docker_err
killall dockerd
rm /var/run/docker.pid 2>/dev/null
set -x
sudo dockerd -b none --iptables=0 -l warn

In [ ]:
!sudo docker pull container-registry.oracle.com/database/express:21.3.0-xe

In [ ]:
!sudo docker images

In [ ]:
!sudo docker export $(docker create container-registry.oracle.com/database/express:21.3.0-xe) > img.tar

In [ ]:
# create img folder that we can chroot in (since docker itself does not work in google colab)
!mkdir img

In [ ]:
!tar -C img  --exclude="dev" -xf img.tar && ls -lah

In [ ]:
%%bash
cd img
sudo bash
rm -Rf proc 2>/dev/null
rm -Rf sys 2>/dev/null
rm -Rf dev 2>/dev/null
mkdir proc
mkdir sys
mkdir dev
sudo mount -t proc /proc proc/
sudo mount --rbind /sys sys/
sudo mount --rbind /dev dev/

In [ ]:
%killbgscripts

All background processes were killed.


# Run oracle express

In [ ]:
%%script bash --bg --out script_out --err script_err
sudo bash
sudo chroot img
export ORACLE_BASE=/opt/oracle
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export ORACLE_BASE_HOME=/opt/oracle/homes/OraDBHome21cXE
export ORACLE_PWD=pwd123456

export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
export PWD_FILE="setPassword.sh"
export RUN_FILE="runOracle.sh"
export START_FILE="startDB.sh"
export CREATE_DB_FILE="createDB.sh"
export SETUP_LINUX_FILE="setupLinuxEnv.sh"
export CHECK_SPACE_FILE="checkSpace.sh"
export CHECK_DB_FILE="checkDBStatus.sh"
export USER_SCRIPTS_FILE="runUserScripts.sh"
export INSTALL_DB_BINARIES_FILE="installDBBinaries.sh"
export INSTALL_RSP="db_inst.rsp"
export CONFIG_RSP="dbca.rsp.tmpl"
export CONF_FILE="oracle-xe-21c.conf"

chown -R oracle:users /opt/oracle
chown -R oracle:users /etc/oratab

cd /opt/oracle/
chmod +x ./$RUN_FILE
su oracle
./$RUN_FILE

# Prepare script files

In [19]:
%%writefile /content/img/script.sql
SELECT uid FROM dual;

Writing /content/img/script.sql


In [20]:
%%writefile /content/img/add_user.sql
CREATE USER john IDENTIFIED BY pwd123456;
GRANT CREATE SESSION TO john;
GRANT CREATE ANY PROCEDURE TO john;
GRANT CREATE ANY TABLE TO john;
GRANT CREATE ANY VIEW TO john;

Writing /content/img/add_user.sql


In [21]:
%%writefile /content/img/add_packages.sql
CREATE OR REPLACE PACKAGE test AS
  PROCEDURE test_proc;
END test;
/

CREATE OR REPLACE PACKAGE BODY test AS
  PROCEDURE test_proc AS
  BEGIN
    NULL;
  END;
END test;
/
quit

Writing /content/img/add_packages.sql


In [22]:
%%writefile /content/img/add_tables.sql
CREATE TABLE TEST_TABLE
(
  test_col VARCHAR(255)
);

quit

Writing /content/img/add_tables.sql


In [23]:
%%writefile /content/img/add_views.sql
CREATE VIEW TEST_VIEW AS
  SELECT * FROM TEST_TABLE;

quit

Writing /content/img/add_views.sql


In [24]:
%%writefile /content/img/select_objects.sql

set colsep ,     -- separate columns with a comma
set pagesize 0   -- No header rows
set trimspool on -- remove trailing blanks
set headsep off  -- this may or may not be useful...depends on your headings.
set linesize 4096   -- X should be the sum of the column widths
set numw 4096       -- X should be the length you want for numbers (avoid scientific notation on IDs)

spool select.csv
SELECT * FROM all_objects WHERE LOWER(OWNER)='john';
spool off

Writing /content/img/select_objects.sql


# Execute script files

In [ ]:
%%bash
sudo chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus sys/pwd123456@//localhost:1521/XE AS SYSDBA @script.sql

In [ ]:
%%bash
sudo chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus sys/pwd123456@//localhost:1521/XEPDB1 AS SYSDBA @add_user.sql

In [ ]:
%%bash
chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_packages.sql
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_tables.sql
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @add_views.sql

In [ ]:
%%bash
chroot img
export ORACLE_HOME=/opt/oracle/product/21c/dbhomeXE
export PATH=$PATH:/opt/oracle/product/21c/dbhomeXE/bin/
export ORACLE_SID=XE
sqlplus john/pwd123456@//localhost:1521/XEPDB1 @select_objects.sql 1> select_1.txt 2> select_2.txt

# Get source of git-remote-oracle

In [25]:
%%bash
git clone https://github.com/TheCrazyT/git-remote-oracle

Cloning into 'git-remote-oracle'...


In [26]:
%%bash
cd git-remote-oracle
git checkout git-remote-oracle
git status
git pull

error: pathspec 'git-remote-oracle' did not match any file(s) known to git


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


Already up to date.


# Install dependencies

In [27]:
!pip install oracledb pexpect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 602.1 kB/s eta 0:00:0000:0100:01


# Test git clone

In [ ]:
%%bash
export PATH=$PATH:$(pwd)/git-remote-oracle/src/git_remote_oracle
export GR_ORACLE_DEBUG=1
export GR_ORACLE_DEBUG_GIT=1

export REL_PATH=$(pwd)/git-remote-oracle/src/git_remote_oracle

echo "#!/bin/bash" > $(pwd)/git-remote-oracle/src/git_remote_oracle/git-remote-oracle
echo python3 $(pwd)'/git-remote-oracle/src/git_remote_oracle/git_remote_oracle.py $@' >> $(pwd)/git-remote-oracle/src/git_remote_oracle/git-remote-oracle

chmod +x ${REL_PATH}/git_remote_oracle.py
chmod +x ${REL_PATH}/git-remote-oracle

rm -Rf JOHN 2>/dev/null

git config --global credential.helper store
echo -e "protocol=oracle\nhost=127.0.0.1\nusername=john\npassword=pwd123456"|git credential approve
git clone oracle://127.0.0.1:1521/XEPDB1/JOHN

# Test git pull

In [ ]:
%%bash
export PATH=$PATH:$(pwd)/git-remote-oracle/src/git_remote_oracle
export GR_ORACLE_DEBUG=1
export GR_ORACLE_DEBUG_GIT=1

cd JOHN
git pull

# List files

In [ ]:
!ls -lah JOHN
!ls -lah JOHN/PACKAGE

total 16K
drwxr-xr-x 4 root root 4.0K Jun 24 15:08 .
drwxr-xr-x 1 root root 4.0K Jun 24 15:08 ..
drwxr-xr-x 8 root root 4.0K Jun 24 15:08 .git
drwxr-xr-x 2 root root 4.0K Jun 24 15:08 PACKAGE
total 12K
drwxr-xr-x 2 root root 4.0K Jun 24 15:08 .
drwxr-xr-x 4 root root 4.0K Jun 24 15:08 ..
-rw-r--r-- 1 root root  210 Jun 24 15:08 TEST.sql


# Stop oracle

In [ ]:
%killbgscripts